In [1]:
import caffe
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

/home/ubuntu/anaconda3/envs/caffe_p35/lib/python3.5/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/caffe_p35/lib/python3.5/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/caffe_p35/lib/python3.5/site-packages/matplotlib/font_manager.py:278: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [2]:
files = pd.read_csv('files.csv', encoding='iso-8859-1', sep=';', error_bad_lines=False, header=None)
files.head(5)[0].values

b'Skipping line 661: expected 1 fields, saw 2\nSkipping line 1162: expected 1 fields, saw 2\nSkipping line 7534: expected 1 fields, saw 2\nSkipping line 9162: expected 1 fields, saw 3\nSkipping line 9445: expected 1 fields, saw 2\nSkipping line 9664: expected 1 fields, saw 2\nSkipping line 10879: expected 1 fields, saw 2\nSkipping line 12082: expected 1 fields, saw 2\nSkipping line 13539: expected 1 fields, saw 2\nSkipping line 16718: expected 1 fields, saw 3\n'


array([ './tier-16-images/1.bp.blogspot.com/-RUXCFQVeBQs/TtPV8sbB7AI/AAAAAAAAAHU/-kA8exbj5AY/s1600/PQAAAGB4wSs9K2FBCPZS-1u_Xsae1a1BRTwijGG-Sjwllbl1QC-4DF8M7NZG_DBE3i0CpqvOXQEMt8zNYoYf1IDX304Am1T1UEsEOXSj2hz82yJQl2gu0PH3rqWq.jpg',
       './tier-16-images/1.bp.blogspot.com/-uLrqNwusOTE/UCOh0-o8VkI/AAAAAAAAAJs/YD01uLJ_wdI/s1600/joao+folho.jpg',
       './tier-16-images/1.bp.blogspot.com/-aFlip7sakEQ/T0ubmfypgJI/AAAAAAAAAFI/zIXNVYLWvuo/s1600/bb3ae3af-f808-1feb.jpg',
       './tier-16-images/1.bp.blogspot.com/-eV-L5yhJK7o/US1MunYdHRI/AAAAAAAAAXk/CSsjQb-vF8o/s1600/choquito.jpg',
       './tier-16-images/1.bp.blogspot.com/-0zLEiEZjY1U/T9CyBOmwVrI/AAAAAAAABdk/nKyCBl1TW6s/s1600/Imagem.jpg'], dtype=object)

In [3]:
deploy_file = 'deploy.prototxt'
caffemodel_file = 'snapshot_iter_74880.caffemodel'

net = caffe.Classifier(deploy_file, caffemodel_file)

In [4]:
import  sys
import  time
    
size = (256,256)
files['prediction'] = ''

start = time.time()

print(len(files))

for idx in files.index:
    file = files.iloc[idx][0]
    
    try:

        image = caffe.io.resize_image(caffe.io.load_image(file.strip()), size)
        image = [(image * 255).astype(np.float)]

        caffe_prob = net.predict(image, oversample=False)
        caffe_prediction = caffe_prob.argmax(axis=1)

        files.iloc[idx]['prediction'] = caffe_prediction[0]

        #print(file.strip(), caffe_prediction[0])
        
    except:
        
        sys.stdout.write('error, ')
        sys.stdout.flush()        
        
    if (idx % 10) == 0:
        end = time.time()
        sys.stdout.write(str(idx) + ' ('+ str(round(end - start,3)) +') , ')
        sys.stdout.flush()
        
    if idx > 100:
        break

16712
0 (2.602) , 10 (8.831) , error, 20 (14.913) , 30 (21.193) , 40 (27.618) , 50 (34.027) , error, error, error, 60 (38.525) , 70 (44.914) , error, 80 (50.755) , error, error, error, error, 90 (54.4) , error, error, error, 100 (59.131) , 

In [5]:
files.to_csv('files_prediction.csv.gz', compression='gzip')